In [1]:
from bs4 import BeautifulSoup
import requests
import psycopg2
from collections import deque
conn = psycopg2.connect(user="PhaCu",
                        password ="0983672100",
                        port="5432",
                        database="tiki")
conn.autocommit = True
cur = conn.cursor()


In [2]:

def create_products_table():
    query = """
        CREATE TABLE IF NOT EXISTS products(
        pro_id VARCHAR(20),
        title VARCHAR (225),
        pro_url TEXT,
        cat_id VARCHAR(20),
        image_url TEXT,
        brand VARCHAR(300),
        review VARCHAR(20),
        tikinow INT,
        fin_price VARCHAR(50),
        old_price VARCHAR(50),
        discount VARCHAR(5)); """
    try:
        cur.execute(query)
    except Exception as err:
        print(f'ERROR: {err}')
create_products_table()

In [3]:
class Products:
    def __init__(self, pro_id, title, pro_url,cat_id, image_url, brand, review, tikinow, fin_price, old_price, discount):
        self.pro_id =       pro_id
        self.title=         title
        self.pro_url=       pro_url
        self.cat_id=        cat_id
        self.image_url=     image_url
        self.brand=         brand
        self.review=        review
        self.tikinow=       tikinow
        self.fin_price=     fin_price
        self.old_price=     old_price
        self.discount=      discount
    def insert_product(self):
        query = f"""INSERT INTO products(pro_id,title,pro_url ,cat_id ,image_url ,brand,review,tikinow ,fin_price ,old_price ,discount)
                    VALUES('{self.pro_id }','{self.title}','{self.pro_url}','{self.cat_id}','{self.image_url}','{self.brand}','{self.review}','{self.tikinow}','{self.fin_price}','{self.old_price}','{self.discount}');"""
        
        try:
            cur.execute(query)
        except Exception as err:
            print(f'ERROR: {err}')
        conn.commit()
    def __repr__(self):
        return f'product: {self.title}, cat_id: {self.cat_id}, Price: {self.fin_price}'

In [7]:
query = """
        SELECT cp.parent_id, cp.name, cp.url, id as child_id FROM cat_parents as cp
         LEFT JOIN categories as c on cp.parent_id = c.parent_id
         WHERE id is null
        ; """
cur.execute(query)
cat=cur.fetchall()

In [8]:
def load_deepest(url):
    try:
        response = requests.get(url).text
        return BeautifulSoup(response,"html.parser")
    except Exception as err:
        print(f'ERROR: {err}')

In [ ]:
for i in range(len(cat)):  
    k=1
    l=0
    while l == 0:
        url=f'{cat[i][2]}&page={k}'
        k=k+1
        try:
            eyes=load_deepest(url).find_all('div', {"class":'product-item'})
        except Exception as err:
            print(f'ERROR:')
        if load_deepest(url) is None:
            l=1
        elif load_deepest(url).find('div', class_='product-box-list').find('div', class_='product-item') is None:
            l=1
        else:
            for j in range (len(eyes)):
                proid=eyes[j]['data-id']
                title=eyes[j]['data-title']
                brand=eyes[j]['data-brand']
                prourl=eyes[j].a['href']
                catid=cat[i][0]
                imageurl=eyes[j].img['src']
                if eyes[j].find('span', class_='rating-content') is None:
                    review = 'No Rating'
                else:
                    review=eyes[j].find('span', class_='rating-content').find('span')['style'].strip('width:').rstrip('%')
                if eyes[j].find('i',class_="tikicon icon-tikinow") is None:
                    tikinow = 0
                else:
                    tikinow = 1
                finprice = eyes[j].find('span', class_='final-price').text.strip().split()[0]
                oldprice = eyes[j].find('span', class_='price-regular').text.strip()
                discount = eyes[j].find('span',class_="sale-tag sale-tag-square").text
                product  = Products(proid,title,prourl,catid,imageurl,brand,review,tikinow,finprice,oldprice,discount)
                product.insert_product()
        conn.commit()

